In [50]:
# holidays library
#!pip install holidays

In [51]:
#Libreries

import pandas as pd
import holidays

## Loading Dataset

In [52]:
dataPath = "../data/raw/household_power_consumption.txt"
dataset = pd.read_csv(dataPath, sep=';', na_values=['nan','?'], low_memory=False)

dataset.head(5)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [53]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    float64
 3   Global_reactive_power  float64
 4   Voltage                float64
 5   Global_intensity       float64
 6   Sub_metering_1         float64
 7   Sub_metering_2         float64
 8   Sub_metering_3         float64
dtypes: float64(7), object(2)
memory usage: 142.5+ MB


### Convert to Datatime formatting

In [54]:
dataset['Datetime'] = pd.to_datetime(dataset['Date'] + ' ' + dataset['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Establecer 'Datetime' como índice
dataset.set_index('Datetime', inplace=True)

# Eliminar columnas originales si ya no son necesarias
dataset.drop(columns=['Date', 'Time'], inplace=True)

In [55]:
dataset.head(5)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


#### Null Values

In [56]:
# Null Values
print("\nNull Values:")
print(dataset.isnull().sum())


Null Values:
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


In [57]:
dataset.dropna(inplace=True)

In [58]:
# Null Values
print("\nNull Values:")
print(dataset.isnull().sum())



Null Values:
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


### Calculation of Unmeasured Active Energy Consumption

The active energy consumed by other unmeasured electrical equipment is calculated using the following formula:

```
(global_active_power * 1000 / 60) - sub_metering_1 - sub_metering_2 - sub_metering_3
```

Where:
- `global_active_power` is in kilowatts (kW).
- Multiplying by 1000 converts kW to watts (W).
- Dividing by 60 estimates the energy consumed per minute.
- `sub_metering_1`, `sub_metering_2`, and `sub_metering_3` represent the energy consumption in watt hour (W/h) of three measured subcircuits.


In [59]:
dataset['unmeasured'] = (dataset['Global_active_power'] * 1000 / 60) - dataset['Sub_metering_1'] - dataset['Sub_metering_2'] - dataset['Sub_metering_3']
dataset.head(5)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,unmeasured
Datetime,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,52.266667
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,72.333333
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,70.566667
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,71.800000
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,43.100000


### Temporal Variable Extraction

To facilitate the analysis of temporal features such as day of the week, weekend status, holidays, season, and time of day, a set of functions has been implemented.

In [60]:
def get_season(date):
    """
        Return: 0 -> 'Winter'
                1 -> 'Spring'
                2 -> 'Summer'
                3 -> 'Autumn'
    """
    Y = date.year
    if date < pd.Timestamp(f'{Y}-03-21'):
        return 0
    elif date < pd.Timestamp(f'{Y}-06-21'):
        return 1
    elif date < pd.Timestamp(f'{Y}-09-21'):
        return 2
    elif date < pd.Timestamp(f'{Y}-12-21'):
        return 3
    else:
        return 0

In [61]:
def time_of_day(hour):
    """
        Return: 0 -> 'Night'
                1 -> 'Morning'
                2 -> 'Afternoon'
                3 -> 'Evening'
    """
    if 0 <= hour < 6:
        return 0
    elif 6 <= hour < 12:
        return 1
    elif 12 <= hour < 18:
        return 2
    else:
        return 3


In [ ]:
def ResamplingDF(df):
    df_rhm = df.resample('H').mean()     # resample by hour with mean
    df_rdm = df.resample('D').mean()     # resample by day with mean
    df_rwm = df.resample('W').mean()     # resample by week with mean
    df_rmm = df.resample('M').mean()     # resample by month with mean

    df_rhm['day_of_week'] = df_rhm.index.dayofweek                                          # Adding day of week resample by hour (0 = monday, 6 = sunday)
    df_rhm['is_weekend'] = df_rhm['day_of_week'].isin([5, 6]).astype(int)            # Adding column to indicate if the day is a weekend (1 = yes, 0 = no)
    fr_holidays = holidays.France()                                                        # Holidays
    df_rhm['is_holiday'] = [date in fr_holidays for date in df_rhm.index.date]  # Adding column to indicate if the day is a holiday (True = yes, False = no)
    df_rhm['is_holiday']  = df_rhm['is_holiday'].astype(int)
    df_rhm['season'] = df_rhm.index.map(get_season)                                  # Adding column to determine the season based on the date
    df_rhm['time_of_day'] = df_rhm.index.hour.map(time_of_day)                       # Adding Column to determine the time of day

    return df_rhm, df_rdm, df_rwm, df_rmm

In [63]:
df_rhm, df_rdm, df_rwm, df_rmm = ResamplingDF(dataset)

df_rhm.head(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,unmeasured,day_of_week,is_weekend,is_holiday,season,time_of_day
Datetime,,,,,,,,,,,,,
2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111,52.992593,5,1,0,3,2
2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667,36.953333,5,1,0,3,3
2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333,38.553889,5,1,0,3,3
2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333,37.692778,5,1,0,3,3
2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667,33.307778,5,1,0,3,3
2006-12-16 22:00:00,2.200133,0.056167,238.760000,9.523333,0.0,0.133333,4.433333,32.102222,5,1,0,3,3
2006-12-16 23:00:00,2.061600,0.071433,240.619667,8.896667,0.0,0.083333,0.000000,34.276667,5,1,0,3,3
2006-12-17 00:00:00,1.882467,0.102433,240.961833,8.126667,0.0,0.466667,0.000000,30.907778,6,1,0,3,0
2006-12-17 01:00:00,3.349400,0.136933,240.448333,14.246667,0.0,25.233333,0.000000,30.590000,6,1,0,3,0


### Saving processed dataset

In [64]:
savePath = "../data/processed/pro_household_power_consumption.csv"
dataset.to_csv(savePath)

# resampled by hour with mean
savePath = "../data/processed/rhm_household_power_consumption.csv"
df_rhm.to_csv(savePath)

# resampled by day with mean
savePath = "../data/processed/rdm_household_power_consumption.csv"
df_rdm.to_csv(savePath)

# resampled by week with mean
savePath = "../data/processed/rwm_household_power_consumption.csv"
df_rwm.to_csv(savePath)

# resampled by month with mean
savePath = "../data/processed/rmm_household_power_consumption.csv"
df_rmm.to_csv(savePath)